In [ ]:
!pip install pinecone
!pip install langchain-pinecone

In [143]:
import os
import time
from dotenv import load_dotenv
import streamlit as st
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders.pdf import PyPDFDirectoryLoader
from langchain_text_splitters.character import RecursiveCharacterTextSplitter
from langchain_text_splitters.character import CharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.chains.question_answering import load_qa_chain
from langchain_groq import ChatGroq
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
# from pinecone import Pinecone
from langchain_pinecone import Pinecone
from pinecone import ServerlessSpec
import warnings
warnings.filterwarnings('ignore')
load_dotenv()


True

In [144]:
api_key = os.getenv("PINECONE_API_KEY")

In [145]:
##Read the Document 
def read_doc(directory):
    file_loader=PyPDFDirectoryLoader(directory)
    documents=file_loader.load()
    return documents


In [146]:
doc=read_doc('uploads/')
doc
len(doc)

1

In [147]:
# cHUNKING Of The Data
def chunk_data(docs,chunk_size=800,chunk_overlap=80):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=chunk_size,chunk_overlap=chunk_overlap)
    chunked_docs=text_splitter.split_documents(docs)
    return chunked_docs

In [148]:
chunked_docs=chunk_data(doc)
chunked_docs

[Document(metadata={'source': 'uploads/Eligible Candidates List_Keywords Studios_PMEC_2025 Batch.pdf', 'page': 0}, page_content='SL.NO. CANDIDATE NAME GENDER COURSE STREAM INSTITUTE NAME EMAIL ADDRESS CONTACT NO.\n1 SUBRAT SETHY Male B.E. / B.TECH CSE PARALA MAHARAJA ENGINEERING COLLEGE subratsethy017@gmail.com 8658132036\n2 ABHIJIT SAMANT Male B.E. / B.TECH CSE PARALA MAHARAJA ENGINEERING COLLEGE lukshi007@gmail.com 7735099393\n3 SHUBHAM PRADHAN Male B.E. / B.TECH CSE PARALA MAHARAJA ENGINEERING COLLEGE shubhampradhan813@gmail.com 8770367753\n4 ABHIJIT MANSINGH Male B.E. / B.TECH CSE PARALA MAHARAJA ENGINEERING COLLEGE abhijitmsing08@gmail.com 8926236278\n5 AYAN KANUNGO Male B.E. / B.TECH CSE PARALA MAHARAJA ENGINEERING COLLEGE aayankanungo@gmail.com 9437851770\n6 ANSUMAN SIMANTA SEKHAR BHUJABALA Male B.E. / B.TECH CSE PARALA MAHARAJA ENGINEERING COLLEGE ansumanbhujabal1@gmail.com 8260964834'),
 Document(metadata={'source': 'uploads/Eligible Candidates List_Keywords Studios_PMEC_2025 

In [149]:
len(chunked_docs)

3

In [150]:
 embeddings = HuggingFaceEmbeddings()

In [151]:
vectors=embeddings.embed_documents("How is me ?")
len(vectors)

11

In [152]:
from pinecone import Pinecone


In [153]:

pc = Pinecone(api_key=api_key)
index_name="budget"

In [154]:
from langchain_pinecone import Pinecone

In [155]:
index = pc.Index(index_name)
# wait a moment for connection
time.sleep(1)

index.describe_index_stats()

{'dimension': 768,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 1}},
 'total_vector_count': 1}

In [156]:
pc.list_indexes()

[
    {
        "name": "budget",
        "dimension": 768,
        "metric": "cosine",
        "host": "budget-ywnft4n.svc.aped-4627-b74a.pinecone.io",
        "spec": {
            "serverless": {
                "cloud": "aws",
                "region": "us-east-1"
            }
        },
        "status": {
            "ready": true,
            "state": "Ready"
        },
        "deletion_protection": "disabled"
    }
]

In [157]:
if index_name not in pc.list_indexes():
    # pinecone.create_index(
    #     name=index_name, 
    #     metric="cosine",
    #     dimension=DIMENSIONS)
    # docsearch = Pinecone.from_documents(docs, embeddings, index_name=index_name)
    print("Please Create Index first")
else:
    docsearch = pc.from_existing_index(doc,embeddings,index_name=index_name)

Please Create Index first


In [158]:

index=Pinecone.from_documents(doc,embeddings,index_name=index_name)

In [159]:
    llm = ChatGroq(
        model="llama-3.1-70b-versatile",
        temperature=0
    )
    chain=load_qa_chain(llm,chain_type="stuff")

In [160]:
## Cosine Similarity Retreive Results from VectorDB
def retrieve_query(query,k=2):
    matching_results=index.similarity_search(query,k=k)
    return matching_results

In [161]:
## Search answers from VectorDB
def retrieve_answers(query):
    doc_search=retrieve_query(query)
    print(doc_search)
    response=chain.run(input_documents=doc_search,question=query)
    return response

In [163]:
our_query = "Details about Abhijit "
answer = retrieve_answers(our_query)
print(answer)

[Document(id='62d753a3-b6b2-4bfd-affe-2cbb42d6c9ce', metadata={'page': 0.0, 'source': 'uploads/Eligible Candidates List_Keywords Studios_PMEC_2025 Batch.pdf'}, page_content='SL.NO. CANDIDATE NAME GENDER COURSE STREAM INSTITUTE NAME EMAIL ADDRESS CONTACT NO.\n1 SUBRAT SETHY Male B.E. / B.TECH CSE PARALA MAHARAJA ENGINEERING COLLEGE subratsethy017@gmail.com 8658132036\n2 ABHIJIT SAMANT Male B.E. / B.TECH CSE PARALA MAHARAJA ENGINEERING COLLEGE lukshi007@gmail.com 7735099393\n3 SHUBHAM PRADHAN Male B.E. / B.TECH CSE PARALA MAHARAJA ENGINEERING COLLEGE shubhampradhan813@gmail.com 8770367753\n4 ABHIJIT MANSINGH Male B.E. / B.TECH CSE PARALA MAHARAJA ENGINEERING COLLEGE abhijitmsing08@gmail.com 8926236278\n5 AYAN KANUNGO Male B.E. / B.TECH CSE PARALA MAHARAJA ENGINEERING COLLEGE aayankanungo@gmail.com 9437851770\n6 ANSUMAN SIMANTA SEKHAR BHUJABALA Male B.E. / B.TECH CSE PARALA MAHARAJA ENGINEERING COLLEGE ansumanbhujabal1@gmail.com 8260964834\n7 IPSITA NAYAK Female B.E. / B.TECH CSE PARALA M

There are two candidates named Abhijit in the given data. 

1. Abhijit Samant:
   - Gender: Male
   - Course: B.E. / B.TECH
   - Stream: CSE
   - Institute Name: PARALA MAHARAJA ENGINEERING COLLEGE
   - Email Address: lukshi007@gmail.com
   - Contact No: 7735099393

2. Abhijit Mansingh:
   - Gender: Male
   - Course: B.E. / B.TECH
   - Stream: CSE
   - Institute Name: PARALA MAHARAJA ENGINEERING COLLEGE
   - Email Address: abhijitmsing08@gmail.com
   - Contact No: 8926236278
